In [1]:
# Install pip packages in the current Jupyter kernel
import sys

!{sys.executable} -m pip install pandas==2.1.4

import pandas as pd

In [2]:
# read data
verses_data = "../data/verses.csv"
gendervoc_data = "../data/names.csv"
occurrences_data = "../data/occurrences.csv"
manuscript_data = "../data/manuscripts.csv"

verses_df = pd.read_csv(verses_data,
    low_memory=False,
    dtype={
        "docID": "Int64",
        "ga": "string",
        "bkv": "string",
        "text": "string",
        "marks": "string",
        "publisher": "string",
        "source": "string",
        "nkv": "string",
        "ntvmrLink": "string",
    },)
gendervoc_df = pd.read_csv(
    gendervoc_data,
    low_memory=False,
    dtype={
        "label:en": "string",
        "label:el": "string",
        "gender": "string",
        "factgrid": "string",
        "variant": "string",
        "wordID": "Int64",
        "variantID": "Int64",
    },
)
occurrences_df = pd.read_csv(
    occurrences_data,
    low_memory=False,
    dtype={
        "docID": "Int64",
        "ga": "string",
        "bkv": "string",
        "text": "string",
        "marks": "string",
        "publisher": "string",
        "source": "string",
        "nkv": "string",
        "ntvmrLink": "string",
        "wordID": "Int64",
        "occurrence": "boolean",
    },
)
manuscript_df = pd.read_csv(manuscript_data,
    low_memory=False,
    dtype={
        "docID": "Int64",
        "originYearLate": "Int64",
        "originYearEarly": "Int64",
        "pagesCount": "Int64",
        "leavesCount": "Int64",
        "ga": "string",
        "century": "string",
        "shelfInstances": "string",
        "source": "string",
        "label": "string",
        "dbpedia": "string",
        "form": "string",
        "found": "string",
    },
)

In [3]:
# drop unnecessary columns
verses_df.drop(columns=["docID","ntvmrLink"], inplace=True)
manuscript_df.drop(
    columns=["form", "shelfInstances", "originYearEarly", "originYearLate", "found"],
    inplace=True,
)

In [4]:
# write dataframes to files
verses_df.to_csv("../data/publish/verses.csv", index=False)
gendervoc_df.to_csv("../data/publish/names.csv", index=False)
occurrences_df.to_csv("../data/publish/occurrences.csv", index=False)
manuscript_df.to_csv("../data/publish/manuscripts.csv", index=False)